In [51]:
%pip install sentiment-analysis-spanish

In [41]:
import pandas as pd
from ntscraper import Nitter

scraper = Nitter(log_level= 0, skip_instance_check=False)
language = ""

Testing instances: 100%|██████████| 16/16 [00:27<00:00,  1.74s/it]


In [42]:

def get_tweets(query: str, mode: str = "term", num_tweets: int = 100, lang = 'es'):
    '''
    Main function for web scraping tweets.
    Args:
        query: Words that are going to be searched
        mode: Searching query as a "term", "hashtag" or a "user" tweet
        num_tweets: Number of tweets to be scraped
        lang: Language of the tweets to be scraped
    '''
    global language
    language = lang
    tweets = scraper.get_tweets(terms= query, mode= mode, number= num_tweets, language= lang)
    return tweets

In [43]:
tweets = get_tweets(query="Madrid", lang="en")

10-Sep-24 09:21:09 - Empty page on https://nitter.privacydev.net


In [44]:
tweets

{'tweets': [], 'threads': []}

In [45]:
final_tweets = []
for tweet in tweets['tweets']:
    data = [tweet['user']['profile_id'], tweet['text']]
    final_tweets.append(data)

In [46]:
data = pd.DataFrame(final_tweets, columns = ['profile_id', 'text'])

In [47]:
import re
from nltk.corpus import stopwords
import nltk
import emoji

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

def clean_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', '', tweet)  # Deletes URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Deletes mentions
    tweet = re.sub(r'#\w+', '', tweet)  # Deletes hashtags
    tweet = re.sub(r'\d+', '', tweet)  # Deletes numbers
      # Converts text into lowcase
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
    tweet = emoji.replace_emoji(tweet, replace= '') # Deletes emojis
    return tweet

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
from pysentimiento import create_analyzer
from langdetect import detect

def get_language(txt: str):
    return detect(txt)
    
def get_sentiment(tweet):
    analyzer = create_analyzer(task="sentiment", lang= language)
    return analyzer.predict(tweet)

for tweet in data['text']:
    cleaned_tweet = clean_tweet(tweet)
    sentiment = get_sentiment(tweet)
    print(f'\nTweet: {cleaned_tweet}\nSentiment: {sentiment}\n --------------------------------')